In [1]:
HOPSWORK_PROJECT_NAME = "taxidemand"

In [2]:
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR

load_dotenv(PARENT_DIR / ".env")

HOPSWORK_API_KEY = os.environ["HOPSWORK_API_KEY"]

In [3]:
from datetime import datetime
import pandas as pd
from src.data import load_raw_data

from_year = 2022
to_year = datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year + 1):
    rides_one_year = load_raw_data(year)
    rides = pd.concat([rides, rides_one_year])

 File 2022-01 already exists
 File 2022-02 already exists
 File 2022-03 already exists
 File 2022-04 already exists
 File 2022-05 already exists
 File 2022-06 already exists
 File 2022-07 already exists
 File 2022-08 already exists
 File 2022-09 already exists
 File 2022-10 already exists
 File 2022-11 already exists
 File 2022-12 already exists
 File 2023-01 already exists
 File 2023-02 already exists
 File 2023-03 already exists
 File 2023-04 already exists
 File 2023-05 already exists
 File 2023-06 already exists
 File 2023-07 already exists
 File 2023-08 already exists
 File 2023-09 already exists
 File 2023-10 already exists
 File 2023-11 already exists
 File 2023-12 already exists
 File 2024-01 already exists
 File 2024-02 already exists
 File 2024-03 already exists
 File 2024-04 already exists
 File 2024-05 already exists
 File 2024-06 already exists
 File 2024-07 already exists
 File 2024-08 already exists
 File 2024-09 already exists
 File 2024-10 already exists
Failed to down

In [4]:
from src.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 263/263 [00:16<00:00, 16.14it/s]


In [5]:
ts_data.shape

(6532920, 3)

In [6]:
import hopsworks
from confluent_kafka import Producer


In [7]:
HOPSWORK_API_KEY

'HIStsugDKH9AIE6Y.ImZGTncDlwIACwi6s4iY14EBNKdOs1rHsDT3MYcQDsjXt02mlZGzyTQLGBWPE6MU'

In [8]:
project = hopsworks.login(
    project = HOPSWORK_PROJECT_NAME,
    api_key_value = HOPSWORK_API_KEY
)

2024-12-28 18:48:04,288 INFO: Initializing external client
2024-12-28 18:48:04,288 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-28 18:48:11,295 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1207467


In [9]:
feature_store = project.get_feature_store()

In [10]:
FEATURE_GROUP_NAME = "time_series_hourly_feature_group"
FEATURE_GROUP_VERSION = 1

In [11]:
feature_group = feature_store.get_or_create_feature_group(
    name = FEATURE_GROUP_NAME,
    version = FEATURE_GROUP_VERSION,
    description = "Time-series data at hourly frequency",
    primary_key = ["pickup_location_id","pickup_hour"],
    event_time = "pickup_hour"
)

In [12]:
feature_group.insert(ts_data, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1207467/fs/1196121/fg/1394303


Uploading Dataframe: 100.00% |██████████| Rows 6532920/6532920 | Elapsed Time: 08:59 | Remaining Time: 00:00


Launching job: time_series_hourly_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1207467/jobs/named/time_series_hourly_feature_group_1_offline_fg_materialization/executions


(Job('time_series_hourly_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)